<a href="https://colab.research.google.com/github/jeong2624/Deep-Learning-for-the-Life-Sciences-coding-transcription/blob/main/Chapter6_Genetics_and_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Genetics and Deep Learning**

해당 Chapter에서는 딥러닝을 활용하여 유전체학에 적용한 사례를 살펴본다.

* 모든 생물은 유전체 (Genome)을 갖고 있으며, 유전체를 구성하는 DNA에는 생물이 살아가는 데 필요한 모든 정보가 들어있음.

* 즉, 세포가 컴퓨터라면 유전체는 실행되는 소프트웨어라고 볼 수 있으며 DNA는 세포에 의해 처리된 정보라고 할 수 있음.

* DNA는 추상적인 저장 매체가 아니라 더 복잡한 방식으로 움직이는 물리적인 분자이며, 또한 수천 개의 다른 분자와 상호작용하며 DNA에 포함된 정보를 유지, 복사, 수행하는 역할을 함.

* 이러한 특징 때문에 여전히 과학자들은 유전체가 어떻게 작동하는지 잘 이해하지 못하고 있음.

* 머신러닝이 유전체 분야에서 강력한 도구로 이용하고 있는데, 유전체 데이터는 엄청난 양의 데이터를 가지고 있고 놀라울 정도로 복잡하고 잘 이해되지 않음에도 불구하고 미묘한 패턴을 발견하는 것이 딥러닝이 잘하는 것이기 때문.

In [1]:
# 필요한 라이브러리 설치
!pip install --pre deepchem
!pip install numpy torch_geometric pytorch_lightning dm-haiku

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 900.7/900.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.0/371.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 17.9 MB/s eta 0:00:00


In [2]:
# 필요한 라이브러리 불러오기
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np

# 재현성을 위해 시드값 배정
np.random.seed(20231110)

In [3]:
# 데이터를 다운받은 후, 로컬 환경에서 구글 코랩에 파일 업로드
# https://github.com/deepchem/DeepLearningLifeSciences/tree/master/Chapter06
from google.colab import files

myfiles = files.upload()

# zip 압축 파일 헤제
!unzip -qq train_dataset.zip
!unzip -qq valid_dataset.zip
!unzip -qq test_dataset.zip
!unzip -qq train_siRNA.zip
!unzip -qq valid_siRNA.zip

Saving accessibility.txt to accessibility.txt
Saving test_dataset.zip to test_dataset.zip
Saving train_dataset.zip to train_dataset.zip
Saving train_siRNA.zip to train_siRNA.zip
Saving valid_dataset.zip to valid_dataset.zip
Saving valid_siRNA.zip to valid_siRNA.zip


### **1. 전사인자의 결합을 예측하는 합성곱 모델**

* 전사인자 JunD를 예시로 해당 전사인자의 결합을 예측하는 모델을 만들어본다.

* 여기서는 인간의 가장 작은 염색체인 22번 염색체의 데이터를 사용하여 학습을 진행.

* 해당 데이터는 101개의 염기로 구성된 DNA 조각들이며, JunD의 결합 여부가 레이블로 포함돼 있음.

In [4]:
# 모델 매개변수 설정

# 각 샘플에 대한 벡터는 염기의 수 (101bp)와 각 염기 (A, T, G, C)에 대한 one-hot encoding의 수를 곱한 404개의 요소로 구성됨.
features = tf.keras.Input(shape = (101, 4)) # 입력값 크기 정의

# 해당 데이터셋은 1% 미만의 데이터만 결합 위치 모티프를 포함하고 있어 매우 불균형함.
# 불균형 현상을 방지하기 위해 데이터셋에 가중치를 추가
prev = features
for i in range(3):
  prev = layers.Conv1D(
      filters = 15, # 각 layer에 몇 개의 filter를 사용할 것인가? -> 15개의 출력값을 생성.
      kernel_size = 10, # 합성곱 커널의 너비 : 10 -> A, T, G, C 염기 개수를 입력값으로 사용해 연속적인 서열을 받아 총 40개 (4 × 10) 의 입력값을 받음.
      activation = tf.nn.relu, # 활성화 함수 설정
      padding = 'same' # 출력 크기를 입력 크기와 동일하게 설정
      )(prev)

  # 과적합 방지하기 위해 Dropout layer 추가
  prev = layers.Dropout(
      rate = 0.5, # 모든 출력값의 절반이 임의로 제거
      )(prev)

# Dense layer를 사용하여 출력값을 계산.
logits = layers.Dense(units = 1)(layers.Flatten()(prev)) # 1개의 출력값을 생성 및 1차원 벡터로 변환
output = layers.Activation(tf.math.sigmoid)(logits) # 시그모이드 함수 적용

In [5]:
# 전사인자의 결합을 예측하는 합성곱 모델 생성
keras_model = tf.keras.Model(
    inputs = features, # 입력값
    outputs = [logits, output] # 출력값 -> 시그모이드 함수 입력값, 결합 부위 존재 확률
)

model = dc.models.KerasModel(
    keras_model, # keras 모델
    loss = dc.models.losses.SigmoidCrossEntropy(), # 손실함수 정의
    output_types = ['prediction', 'loss'], # 출력값 -> 결합 부위 존재 여부, 교차 엔트로피 값
    batch_size = 1000, # 모델 학습 중 parameter 업데이트를 할 때 몇 개의 데이터를 사용할 것인가?
    model_dir = "tf"
)

In [6]:
# 학습 데이터 및 검정 데이터 불러오기
train = dc.data.DiskDataset('train_dataset')
valid = dc.data.DiskDataset('valid_dataset')

In [7]:
# 모델 학습 및 평가
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
for i in range(20):
  model.fit(train, nb_epoch = 10)
  print(model.evaluate(train, [metric]))
  print(model.evaluate(valid, [metric]))

{'roc_auc_score': 0.5329209490034716}
{'roc_auc_score': 0.5565352876887641}
{'roc_auc_score': 0.5316914601493716}
{'roc_auc_score': 0.5448513494830093}
{'roc_auc_score': 0.531634615307993}
{'roc_auc_score': 0.5442162424053107}
{'roc_auc_score': 0.5278478494339021}
{'roc_auc_score': 0.5341936829556624}
{'roc_auc_score': 0.5276223875419204}
{'roc_auc_score': 0.5341354116820656}
{'roc_auc_score': 0.5273304459709747}
{'roc_auc_score': 0.5360795804308927}
{'roc_auc_score': 0.5268546611776953}
{'roc_auc_score': 0.536095816700288}
{'roc_auc_score': 0.5258614931545698}
{'roc_auc_score': 0.5359920241069753}
{'roc_auc_score': 0.5257513395918342}
{'roc_auc_score': 0.5361042834665372}
{'roc_auc_score': 0.5256021081675208}
{'roc_auc_score': 0.5361008967600376}
{'roc_auc_score': 0.5255469870290137}
{'roc_auc_score': 0.5361275919759759}
{'roc_auc_score': 0.5255230279481977}
{'roc_auc_score': 0.5361251017506086}
{'roc_auc_score': 0.525471039824481}
{'roc_auc_score': 0.5361257990137114}
{'roc_auc_score

### **2. 염색질 접근성 정보를 추가하여 전사인자의 결합을 예측하는 합성곱 모델**

* 염색질은 염색체를 구성하는 단위로 DNA, 히스톤, RNA로 구성된 거대 분자 복합체.

* 염색질 접근성은 염색체의 각 부분이 외부 분자에 얼마나 쉽게 접근할 수 있는지를 나타냄.

* DNA가 히스톤 주변에 단단히 감겨 있으면 전사인자와 다른 분자가 접근할 수 없음.

* 염색질 접근성은 세포가 유전자의 발현을 조절하는 데 사용하는 도구로 균일하지 않고 정적이지 않음. 이는 세포의 유형과 세포 주기에 따라 매우 달라지며 환경적인 영향을 받기도 하다는 의미.

* 앞서 분석한 JunD 전사인자 데이터셋은 HepG2 (간세포암종 세포주) 세포주를 사용한 실험을 통해 얻은 것이며, 해당 세포주의 염색질 접근성에 의해 실제 전사인자 결합 부위가 달라졌을 수 있음.

따라서, 이전 머신러닝 모델에 염색질 접근성 정보를 추가하는 작업을 해보자.

In [8]:
# 모델 매개변수 설정

# 입력값 크기 정의
features = tf.keras.Input(shape = (101, 4))
accessibility = tf.keras.Input(shape = (1, ))

# 해당 데이터셋은 1% 미만의 데이터만 결합 위치 모티프를 포함하고 있어 매우 불균형함.
# 불균형 현상을 방지하기 위해 데이터셋에 가중치를 추가
prev = features
for i in range(3):
  prev = layers.Conv1D(
      filters = 15, # 각 layer에 몇 개의 filter를 사용할 것인가?
      kernel_size = 10, # 합성곱 커널의 너비
      activation = tf.nn.relu, # 활성화 함수 설정
      padding = 'same' # 출력 크기를 입력 크기와 동일하게 설정
      )(prev)

  # 과적합 방지하기 위해 Dropout layer 추가
  prev = layers.Dropout(
      rate = 0.5,
      )(prev)

# 두 개의 입력 벡터 합치기 및 1개의 출력값을 생성 및 1차원 벡터로 변환
prev = layers.Concatenate()([layers.Flatten()(prev), accessibility])
logits = layers.Dense(units = 1)(prev) # Dense layer를 사용하여 출력값을 계산.
output = layers.Activation(tf.math.sigmoid)(logits) # 시그모이드 함수 적용

In [9]:
# 전사인자의 결합을 예측하는 합성곱 모델 생성
keras_model = tf.keras.Model(
    inputs = [features, accessibility], # 입력값
    outputs = [logits, output] # 출력값 -> 시그모이드 함수 입력값, 결합 부위 존재 확률
)

model = dc.models.KerasModel(
    keras_model,
    loss=dc.models.losses.SigmoidCrossEntropy(), # 손실함수 정의
    output_types = ['prediction', 'loss'], # 출력값
    batch_size = 1000, # 모델 학습 중 parameter 업데이트를 할 때 몇 개의 데이터를 사용할 것인가?
    model_dir = 'chromatin'
    )

In [10]:
# 학습 데이터 및 검정 데이터 불러오기
train = dc.data.DiskDataset('train_dataset')
valid = dc.data.DiskDataset('valid_dataset')

# 염색질 접근성 데이터 불러오기
span_accessibility = {}
for line in open('accessibility.txt'):
    fields = line.split()
    span_accessibility[fields[0]] = float(fields[1]) # 염색체 부위 식별자 (i.g. chr22:20208963-20209064): 염색질 접근성 수치

In [11]:
# feature layer가 두 개 있기 때문에 fit() 함수를 사용할 수 없음.
# 이를 해결하기 위해, 생성자 함수를 통해 배치별 반복 학습 수행
def generate_batches(dataset, epochs):
    for epoch in range(epochs):
        for X, y, w, ids in dataset.iterbatches(batch_size = 1000, pad_batches = True):
            yield ([X, np.array([span_accessibility[id] for id in ids])], [y], [w])

In [12]:
# 모델 학습 및 평가
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
for i in range(20):
    model.fit_generator(generate_batches(train, epochs = 10))
    print(model.evaluate_generator(generate_batches(train, 1), [metric]))
    print(model.evaluate_generator(generate_batches(valid, 1), [metric]))

{'roc_auc_score': 0.5427153362745221}
{'roc_auc_score': 0.5623695635256581}
{'roc_auc_score': 0.6116264378488627}
{'roc_auc_score': 0.5829681970349115}
{'roc_auc_score': 0.6533903928446769}
{'roc_auc_score': 0.6059582413288387}
{'roc_auc_score': 0.6622667367050801}
{'roc_auc_score': 0.5974617405993377}
{'roc_auc_score': 0.6936092917429748}
{'roc_auc_score': 0.6139982660268439}
{'roc_auc_score': 0.7105355366016542}
{'roc_auc_score': 0.6276060848879381}
{'roc_auc_score': 0.719656119100886}
{'roc_auc_score': 0.6268496923762608}
{'roc_auc_score': 0.7291106387539031}
{'roc_auc_score': 0.6459600682438642}
{'roc_auc_score': 0.7259571008811398}
{'roc_auc_score': 0.6314070114890704}
{'roc_auc_score': 0.7360047035901742}
{'roc_auc_score': 0.6501073755021927}
{'roc_auc_score': 0.739049179369236}
{'roc_auc_score': 0.6511555340267443}
{'roc_auc_score': 0.7421610032655375}
{'roc_auc_score': 0.6529076996652319}
{'roc_auc_score': 0.742144361522221}
{'roc_auc_score': 0.6458361268933017}
{'roc_auc_score

### **3. RNA 간섭**

* RNA 간섭은 작은 RNA 조각 (siRNA)이 상보적인 mRNA 서열에 결합한 후 비활성화해서 단백질로의 번역을 막는 현상.

* RNA 간섭은 복잡한 생물학적 과정이며 붙어있는 두 가닥의 RNA 작용뿐만 아니라 그보다 더 많은 과정이 포함됨. (유전자 발현 조절, 바이러스에 대한 방어 기작)

* 해당 데이터셋은 각각 21개의 염기로 구성된 총 2,431개의 siRNA로 구성되어 있으며, 해당 siRNA들은 모두 실험적으로 효과가 검증됐고 0에서 1 사이의 값으로 표현. (값이 1에 가까울수록 RNA 간섭이 잘 일어남을 의미)

따라서, RNA 서열을 입력 데이터로 사용해 RNA 간섭 효과를 예측하는 모델을 만들어본다.

In [13]:
# 모델 매개변수 설정

# 입력값 크기 정의
features = tf.keras.Input(shape = (21, 4))

# 불균형 현상을 방지하기 위해 데이터셋에 가중치를 추가
prev = features
for i in range(2):
  prev = layers.Conv1D(
      filters = 10, # 각 layer에 몇 개의 filter를 사용할 것인가?
      kernel_size = 10, # 합성곱 커널의 너비
      activation = tf.nn.relu, # 활성화 함수 설정
      padding = 'same' # 출력 크기를 입력 크기와 동일하게 설정
      )(prev)

  # 과적합 방지하기 위해 Dropout layer 추가
  prev = layers.Dropout(
      rate = 0.3,
      )(prev)

# 출력값 정의
output = layers.Dense(units = 1, activation = tf.math.sigmoid)(layers.Flatten()(prev))

In [14]:
# RNA 간섭을 예측하는 합성곱 모델 생성
keras_model = tf.keras.Model(
    inputs = features,
    outputs = output)

model = dc.models.KerasModel(
    keras_model, # keras 모델
    loss = dc.models.losses.L2Loss(), # 손실함수 정의
    batch_size = 1000, # 모델 학습 중 parameter 업데이트를 할 때 몇 개의 데이터를 사용할 것인가?
    model_dir = "rnai"
)

In [15]:
# 학습 데이터 및 검정 데이터 불러오기
train = dc.data.DiskDataset('train_siRNA')
valid = dc.data.DiskDataset('valid_siRNA')

# 모델 학습 및 평가
metric = dc.metrics.Metric(dc.metrics.pearsonr, mode = 'regression')
for i in range(20):
  model.fit(train, nb_epoch = 10)
  print(model.evaluate(train, [metric])['pearsonr'])
  print(model.evaluate(valid, [metric])['pearsonr'])

0.14605836901023783
0.21526241293547493
0.3940111615017957
0.39498553394051356
0.4906341519132713
0.4472834886588448
0.5538875726206277
0.48439236020086024
0.6002988680741739
0.5261953201164888
0.6238535925070778
0.5468435946696889
0.643089038193633
0.565987928106023
0.6578168649680985
0.5788285335011547
0.6710699775815255
0.5923505623760502
0.6782130023104167
0.5976205366008043
0.6872070737250024
0.6012548747130301
0.6924556526633852
0.6049670250862738
0.6969174309313387
0.60904086671645
0.7015255421083213
0.6111383435001315
0.7070176765948372
0.6153656093096584
0.7088239777569179
0.6127246711722698
0.713123099238385
0.6147918159158182
0.7161899341148075
0.6160870840854705
0.7185696646186188
0.6139272864451167
0.7222633572680459
0.6132713895464732
